In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics

# load the data
train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

# convert time column to datetime format
train_df['time'] = pd.to_datetime(train_df['time'])
test_df['time'] = pd.to_datetime(test_df['time'])

# extract hour and minute components
train_df['hour'] = train_df['time'].dt.hour
train_df['minute'] = train_df['time'].dt.minute
test_df['hour'] = test_df['time'].dt.hour
test_df['minute'] = test_df['time'].dt.minute

# one-hot encode the direction feature
train_direction = pd.get_dummies(train_df['direction'], prefix='dir')
test_direction = pd.get_dummies(test_df['direction'], prefix='dir')
train_df = pd.concat([train_df, train_direction], axis=1)
test_df = pd.concat([test_df, test_direction], axis=1)

# define features and target variable
features = ['hour', 'minute', 'x', 'y', 'dir_EB', 'dir_NB', 'dir_SB', 'dir_WB']
target = 'congestion'

# split the data into training and validation sets
train_features, val_features, train_target, val_target = train_test_split(
    train_df[features], train_df[target], test_size=0.2, random_state=42)

# train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(train_features, train_target)
y_pred=model.predict(val_features)

# make predictions on the test set
test_predictions = model.predict(test_df[features])

# create submission file
submission_df = pd.DataFrame({'row_id': test_df['row_id'], 'congestion': test_predictions})
submission_df.to_csv('submission.csv', index=False)

In [ ]:
print("mean absolute error",metrics.mean_absolute_error(val_target,y_pred))
print("mean squared error",metrics.mean_squared_error(val_target,y_pred))
print("root mean squared error",np.sqrt(metrics.mean_squared_error(val_target,y_pred)))

mean absolute error 7.832284094457078
mean squared error 113.63295834016226
root mean squared error 10.659876094034221


In [ ]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 49.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 K

In [ ]:
import gradio as gr

def congestion_prediction(hour, minute, x, y, dir_EB, dir_NB, dir_SB, dir_WB):
    # create a dictionary of the input features
    input_dict = {
        'hour': [hour],
        'minute': [minute],
        'x': [x],
        'y': [y],
        'dir_EB': [dir_EB],
        'dir_NB': [dir_NB],
        'dir_SB': [dir_SB],
        'dir_WB': [dir_WB]
    }
    
    # create a dataframe from the input dictionary
    input_df = pd.DataFrame.from_dict(input_dict)
    
    # make a prediction using the trained model
    prediction = model.predict(input_df)
    
    # return the predicted congestion value
    return prediction[0]

# create a Gradio interface
input_interface = gr.Interface(congestion_prediction, 
                               inputs=['number', 'number', 'number', 'number', 'checkbox', 'checkbox', 'checkbox', 'checkbox'], 
                               outputs='number',
                               title='Traffic Flow Forecast',
                               description='Enter the input features to predict the congestion level.')

# launch the interface
input_interface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>